<a href="https://colab.research.google.com/github/Saransh-Bhatnagar/Resume_Classification/blob/main/CV_Assgnment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image
import os
import numpy as np
import cv2

import random

from sklearn.model_selection import train_test_split


In [2]:
!mkdir resume
!mkdir ext

In [3]:
(train_ds, _), (_, _) = tf.keras.datasets.cifar10.load_data()

# Get 60 random images
random_indices = tf.random.uniform((60,), minval=0, maxval=train_ds.shape[0], dtype=tf.int32)
random_images = train_ds[random_indices]

170498071/170498071 [==============================] - 6s 0us/step


In [4]:
random_images.shape

(60, 32, 32, 3)

In [5]:
directory_path = '/content/ext'

extra = []  # List to store image arrays

# Iterate through the directory
for filename in os.listdir(directory_path):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        img_path = os.path.join(directory_path, filename)
        img = Image.open(img_path)
        img_array = np.array(img)
        extra.append(img_array)

In [6]:
resized_images = []

# Resizing to match the required shape (32x32x3)
for img in extra:
    # Resize each image to a common shape (e.g., 32x32x3)
    resized_img = cv2.resize(img, (32, 32))
    resized_images.append(resized_img)

# Convert the list of resized images to a NumPy array
rextra = np.array(resized_images)

In [7]:
random_images = np.concatenate((random_images,rextra), axis=0)

In [8]:
random_images.shape

(79, 32, 32, 3)

In [9]:
directory_path = '/content/resume'

image_arrays = []  # List to store image arrays

# Iterating through the directory
for filename in os.listdir(directory_path):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        img_path = os.path.join(directory_path, filename)
        img = Image.open(img_path)
        img_array = np.array(img)
        image_arrays.append(img_array)


In [10]:
type(image_arrays)

list

In [11]:
labels=[1 for x in image_arrays]
y2=[0 for x in random_images]

In [12]:
for i in y2:
  labels.append(i)

In [13]:
for i in random_images:
  image_arrays.append(i)

image_arrays=np.array(image_arrays,dtype='object')


In [14]:
image_arrays.shape

(156,)

In [15]:
def preprocess(images,target_size = (224, 224)):

  processed_images = [cv2.resize(img, target_size) for img in images]
  processed_images = np.array(processed_images) / 255.0

  return processed_images

In [16]:
processed_images=preprocess(image_arrays)

In [17]:
from sklearn.utils import shuffle
# Shuffling images and labels together
processed_images, labels = shuffle(processed_images, labels)


In [18]:
train_images, val_images, train_labels, val_labels = train_test_split(processed_images, labels, test_size=0.15, random_state=42)

# Ensure shapes for compatibility with the model
train_images = np.array(train_images)
val_images = np.array(val_images)
train_labels = np.array(train_labels)
val_labels = np.array(val_labels)

In [19]:
train_labels = tf.cast(train_labels, dtype=tf.float32)
val_labels = tf.cast(val_labels, dtype=tf.float32)

In [20]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D,Dropout, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy, F1Score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [21]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', Precision(), Recall(), F1Score()])


In [22]:
batch_size = 32

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.15  #
)

# Preparing data generators
train_generator = datagen.flow(train_images, train_labels, batch_size=batch_size, subset='training')
val_generator = datagen.flow(val_images, val_labels, batch_size=batch_size, subset='validation')


# Training the model using the generators
model.fit(train_generator,  epochs=10, validation_data=val_generator, batch_size=batch_size)



Epoch 1/10
4/4 [==============================] - 20s 5s/step - loss: 2.1406 - accuracy: 0.4956 - precision: 0.5000 - recall: 0.5965 - f1_score: 0.6706 - val_loss: 0.8164 - val_accuracy: 0.3333 - val_precision: 0.3333 - val_recall: 1.0000 - val_f1_score: 0.5000
Epoch 2/10
4/4 [==============================] - 14s 4s/step - loss: 0.6626 - accuracy: 0.6549 - precision: 0.6452 - recall: 0.7018 - f1_score: 0.6706 - val_loss: 0.6099 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000 - val_f1_score: 0.5000
Epoch 3/10
4/4 [==============================] - 14s 3s/step - loss: 0.6148 - accuracy: 0.6903 - precision: 0.6774 - recall: 0.7368 - f1_score: 0.6706 - val_loss: 0.5029 - val_accuracy: 0.6667 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_f1_score: 0.5000
Epoch 4/10
4/4 [==============================] - 13s 4s/step - loss: 0.5365 - accuracy: 0.7611 - precision: 0.7143 - recall: 0.8772 - f1_score: 0.6706 - val_loss: 0.2903 - val_accuracy: 1.0000 - val_precis

In [23]:
imgtest=[]
img=cv2.imread('/content/ext/cute-baby-animals-1558535060.jpg')
imgtest.append(img)



imgtest=np.array(imgtest)
test=preprocess(imgtest)

In [24]:
test.shape

(1, 224, 224, 3)

In [25]:
prediction = model.predict(test)  # Make predictions
predicted_class = "Resume" if prediction > 0.5 else "Non-Resume"  # Assuming binary classification with threshold 0.5

print(prediction)
print(f"The predicted class is: {predicted_class}")

1/1 [==============================] - 0s 140ms/step
[[0.15470485]]
The predicted class is: Non-Resume
